In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile 
import kaggle   
from geopy.geocoders import Nominatim
from functools import partial
from geopy.extra.rate_limiter import RateLimiter


# https://www.kaggle.com/datasets/reihanenamdari/toronto-bus-delay-2022/

In [4]:
# !kaggle datasets download -d reihanenamdari/toronto-bus-delay-2022

# zipfile_name = 'toronto-bus-delay-2022.zip'
# with zipfile.ZipFile(zipfile_name, 'r') as file:
#     file.extractall()


  0%|          | 0.00/454k [00:00<?, ?B/s]
100%|██████████| 454k/454k [00:00<00:00, 4.18MB/s]
100%|██████████| 454k/454k [00:00<00:00, 4.18MB/s]


In [7]:
ttc_delay = pd.read_csv('ttc-bus-delay-data-2022.csv')

ttc_delay.head()

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,1-Jan-22,320,02:00,Saturday,YONGE AND DUNDAS,General Delay,0,0,NaN,8531
1,1-Jan-22,325,02:00,Saturday,OVERLEA AND THORCLIFFE,Diversion,131,161,W,8658
2,1-Jan-22,320,02:00,Saturday,YONGE AND STEELES,Operations - Operator,17,20,S,0
3,1-Jan-22,320,02:07,Saturday,YONGE AND STEELES,Operations - Operator,4,11,S,0
4,1-Jan-22,320,02:13,Saturday,YONGE AND STEELES,Operations - Operator,4,8,S,0


In [20]:
# get basic description of dataset

ttc_delay.describe()

,Min Delay,Min Gap,Vehicle
count,27351.000000,27351.000000,27351.000000
mean,20.298417,32.876714,5410.169427
std,49.636736,51.293913,4616.285325
min,0.000000,0.000000,0.000000
25%,9.000000,17.000000,1394.000000
50%,11.000000,22.000000,7940.000000
75%,20.000000,36.000000,8523.000000
max,999.000000,999.000000,93561.000000


In [19]:
# check for na before applying lat/long

any(ttc_delay['Location'].isna())

False

In [58]:
def get_coord(location):
    url = 'https://nominatim.openstreetmap.org/search?q='+location.replace(' ', '+') + '&format=json'
    print(url)
    response = requests.get(url).json()
    if not response:
        return [np.nan,np.nan]
    return [response[0]['lat'], response[0]['lon']]

In [60]:
df = pd.DataFrame()
df['location'] = ttc_delay.Location[:100].astype(str) + ', Toronto, Ontario'

In [61]:
df['coords'] = df.location.apply(get_coord)

https://nominatim.openstreetmap.org/search?q=YONGE+AND+DUNDAS,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=OVERLEA+AND+THORCLIFFE,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=YONGE+AND+STEELES,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=YONGE+AND+STEELES,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=YONGE+AND+STEELES,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=KING+AND+SHAW,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=HUMBERLINE+LOOP,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=STEELES+AND+YONGE,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=YONGE+AND+STEELES,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=KENNEDY+STATION,+Toronto,+Ontario&format=json
https://nominatim.openstreetmap.org/search?q=BLOOR+AND+INDIAN,+Toronto,+On

In [78]:
df[:50]

,location,coords
0,"YONGE AND DUNDAS, Toronto, Ontario","[43.6566724, -79.3804743]"
1,"OVERLEA AND THORCLIFFE, Toronto, Ontario","[nan, nan]"
2,"YONGE AND STEELES, Toronto, Ontario","[nan, nan]"
3,"YONGE AND STEELES, Toronto, Ontario","[nan, nan]"
4,"YONGE AND STEELES, Toronto, Ontario","[nan, nan]"
5,"KING AND SHAW, Toronto, Ontario","[nan, nan]"
6,"HUMBERLINE LOOP, Toronto, Ontario","[43.7473391, -79.6223789]"
7,"STEELES AND YONGE, Toronto, Ontario","[nan, nan]"
8,"YONGE AND STEELES, Toronto, Ontario","[nan, nan]"
9,"KENNEDY STATION, Toronto, Ontario","[43.732429, -79.2637529]"
